## Using statistical models to assess the impact of weather conditions on bicycling behavior

In [13]:
%%capture

# All Libraries needed for Regression imported
import pandas as pd
import matplotlib.pyplot as plt
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_columns', 999)
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col
import pandas as pd
from statsmodels.graphics.api import abline_plot
from patsy import dmatrices
import statsmodels.formula.api as smf
import numpy as np
import math

# Reading DFs as CSV files needed for regression
bike_journey_data=pd.read_csv('bike_journey_data.csv')
weather_data=pd.read_csv('weather_data_27_mar_2020.csv')

In [14]:
%%capture
#Collapsing bike journeys by hour and day (24 hours x 365 days) 
bike_journeys=bike_journey_data.loc[bike_journey_data['Duration in minutes'] < 300]
bike_journeys['c'] = 1
daily_bike_journeys = bike_journeys.loc[:, ('c', 'id_hours', 'Duration in minutes')].groupby(['id_hours']).sum()
daily_bike_journeys.reset_index(inplace=True)
daily_bike_journeys.rename(columns={'c': 'Biketrips', 'id_hours': 'id'}, inplace=True)
#print (daily_bike_journeys.shape)

#Merging bike journeys with the weather data by hour and day (24 hours x 365 days) 
reg_data= daily_bike_journeys.merge(right=weather_data,
                             left_on = 'id',
                             right_on = 'id_Hours')

#Generates one of the dependent variables "Avg trip duration"/January 2019 is used as a reference group
reg_data['Avg trip duration']=reg_data['Duration in minutes']/reg_data['Biketrips']

#Generates dummy variables for months of the year to control for seasonality in the regression 
df_month=pd.get_dummies(reg_data.Month, drop_first=True)
df_month.columns=['February','March', 'April','May','June','July',
                  'August','September','October','November','December']
reg_data = reg_data.join(df_month)

#Creates dummy variables for temperature by cutting it into 5°C bins/Temp - 15(°C) is used as a reference group
reg_data["Temp_0"] = reg_data['temperature'].apply(lambda x: 1 if x <=0 else 0)
reg_data["Temp_5"] = reg_data['temperature'].apply(lambda x: 1 if x>0 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=5 else 0)
reg_data["Temp_10"] = reg_data['temperature'].apply(lambda x: 1 if x>5 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=10 else 0)
#reg_data["Temp - 15(°C)"] = reg_data['temperature'].apply(lambda x: 1 if x>10 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=15 else 0)
reg_data["Temp_20"] = reg_data['temperature'].apply(lambda x: 1 if x>15 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=20 else 0)
reg_data["Temp_25"] = reg_data['temperature'].apply(lambda x: 1 if x>20 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=25 else 0)
reg_data["Temp_30"] = reg_data['temperature'].apply(lambda x: 1 if x>25 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=30 else 0)#reg_data["temp_35"] = reg_data['temperature'].apply(lambda x: 1 if x>30 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=35 else 0)
reg_data["Temp_35"] = reg_data['temperature'].apply(lambda x: 1 if x>30 else 0) & reg_data['temperature'].apply(lambda x: 1 if x<=38 else 0)

#Creates dummy variables for humidity, rain, wind speed, visibility, weekend, holidays, peak travel hours
reg_data["Humidity"] = reg_data['humidity'].apply(lambda x: 1 if x >=0.6497 else 0)
reg_data["Rain"] = reg_data['precipIntensity'].apply(lambda x: 1 if x >0.0697 else 0)
reg_data["Wind"] = reg_data['windSpeed'].apply(lambda x: 1 if x >8.1400 else 0)
reg_data["Visibility"] = reg_data['visibility'].apply(lambda x: 1 if x >15.4914 else 0)
reg_data["Weekend"] = reg_data['Week Day'].apply(lambda x: 1 if x >4 else 0)
reg_data["Peaktravel"] = reg_data['Hours'].apply(lambda x: 1 if x >=7 and x<10 else (1 if x >= 16 and x < 20 else 0))
#reg_data=reg_data.set_index('id')
holidays = ['2019-1-1', '2019-4-19', '2019-4-22', '2019-5-6', '2019-5-27', '2019-8-26', '2019-12-25', '2019-12-26']
reg_data['Holidays'] = reg_data.index.get_level_values(0).isin(holidays).astype(int)

In [15]:
%%capture
#Collapsing bike journeys by hour and day (24 hours x 365 days) 
bike_journeys=bike_journey_data.loc[bike_journey_data['Duration in minutes'] < 300]
bike_journeys['c'] = 1
daily_bike_journeys = bike_journeys.loc[:, ('c', 'id_hours','Duration in minutes')].groupby(['id_hours']).sum()
daily_bike_journeys.reset_index(inplace=True)
#daily_bike_journeys.rename(columns={'c': 'Biketrips', 'id_hours': 'id'}, inplace=True)

In [16]:
%%capture
#Generates descriptive statistics for key dependent and independent variables in the regression

# Trips per hour
Trips_hour = reg_data['Biketrips'].describe()
Trips_hour

# Trip duration per hour
Avg_trip_duration = reg_data['Avg trip duration'].describe()
Avg_trip_duration

# Teperature
Teperature = reg_data['temperature'].describe()
Teperature

# Humidity
Humidity = reg_data['Humidity'].describe()
Humidity

# Rain
Rain = reg_data['Rain'].describe()
Rain

# Wind Speed
Wind= reg_data['Wind'].describe()
Wind

# Visibility
Visibility= reg_data['Visibility'].describe()
Visibility

# Peak Travel Hours
Peak_travel_hours = reg_data['Peaktravel'].describe()
Peak_travel_hours

#Weekends
Weekend = reg_data['Weekend'].describe()
Weekend

#Holidays
Holidays = reg_data['Holidays'].describe()
Holidays

In [17]:
%%capture
#Set up the dependent and independent variables and run the OLS regression

y1 = 'Avg trip duration'
y2 = 'Biketrips'

x = ['Temp_0','Temp_5','Temp_10','Temp_20','Temp_25','Temp_30','Temp_35',
                         'Humidity','Rain','Wind','February','March','April','May','June','July','August',
                          'September','October','November','December','Peaktravel','Holidays','Weekend',
                          'Visibility']

#Run the OLS regression model
regression_results = sm.OLS(reg_data[y1], sm.add_constant(reg_data[x])).fit()

# Outputing the regression results
dfoutput = summary_col([regression_results],stars=True, 
                       regressor_order=['Temp_0','Temp_5','Temp_10','Temp_20','Temp_25','Temp_30','Temp_35',
                         'Humidity','Rain','Wind','February','March','April','May','June','July','August',
                          'September','October','November','December','Peaktravel','Holidays','Weekend',
                          'Visibility'])

#dfoutput.add_title('Table 1: OLS Model for Weather Impacts on Bicycling Behavior')

print(dfoutput)

In [18]:
%%capture
# Poisson Regression Dataframe
data=reg_data[['Biketrips', 'Temp_0','Temp_5','Temp_10','Temp_20','Temp_25','Temp_30','Temp_35',
                         'Humidity','Rain','Wind','February','March','April','May','June','July','August',
                          'September','October','November','December','Peaktravel','Holidays','Weekend',
                         'Visibility']]

#Divide the dataframe into a train and test dataframe
mask = np.random.rand(len(data)) < 0.8
data_train = data[mask]
data_test = data[~mask]
#print('Training data set length='+str(len(poisson_dataset_train)))
#print('Testing data set length='+str(len(poisson_dataset_test)))

#Setup the Poisson regression expression in patsy notation. We are telling patsy that Biketrips is our dependent variable and it depends on the other categorical independent regression variables
formula = """Biketrips ~ C(Temp_0)  + C(Temp_5) + C(Temp_10) + C(Temp_20) + C(Temp_25) + C(Temp_30)
                + C(Temp_35) + C(Humidity) + C(Rain) + C(Wind) +C(February)+C(March)+C(April)
                +C(May)+C(June)+C(July)+C(August)+C(September)+C(October)+C(November)+C(December)
                +C(Peaktravel)+C(Holidays)+C(Weekend)+C(Visibility)"""

#Set up the X and Y matrices for the training and testing data sets
y_train, X_train = dmatrices(formula, data_train, return_type='dataframe')

#Train the Poisson regression model on the training data set
results = sm.GLM(y_train, X_train, family=sm.families.Poisson()).fit()
print(results.summary())

#Print the fitted mean of bike trips 
print(results.mu)
print(len(results.mu))

#Print the model Pearson residuals
print(results.resid_pearson)
print(len(results.resid_pearson))

In [19]:
%%capture
#Plot the fitted values vs observed values from the Poisson regression
nobs = results.nobs
y = data_train.Biketrips
yhat = results.mu

fig, ax = plt.subplots()
ax.scatter(yhat, y)
line_fit = sm.OLS(y, sm.add_constant(yhat, prepend=True)).fit()
abline_plot(model_results=line_fit, ax=ax)

ax.set_title('Model Fit Plot')
ax.set_ylabel('Observed values')
ax.set_xlabel('Fitted values');

In [20]:
%%capture
#Plot the fitted mean of bike trip counts versus the Pearson residuals for the train data/Poisson regression
fig, ax = plt.subplots()
ax.scatter(yhat, results.resid_pearson)

ax.set_title('Residual vs Mean')
ax.set_ylabel('Pearson residuals')
ax.set_xlabel('Fitted means')

In [21]:
%%capture
#Dispersion calculation for Poisson regression (Alpha parameter)
def ct_response(row):
    "Calculate response observation for Cameron-Trivedi dispersion test"
    y = row['Biketrips']
    m = row['Biketrips_mu']
    return ((y - m)**2 - y) / m

ct_data = data_train.copy()
ct_data['Biketrips_mu'] = results.mu
ct_data['ct_resp'] = ct_data.apply(ct_response, axis=1)

# Linear regression of auxiliary formula
ct_results = smf.ols('ct_resp ~ Biketrips_mu - 1', ct_data).fit()

# Construct confidence interval for alpha, the coefficient of bev_mu
alpha_ci95 = ct_results.conf_int(0.05).loc['Biketrips_mu']
print('\nC-T dispersion test: alpha = {:5.3f}, 95% CI = ({:5.3f}, {:5.3f})'
        .format(ct_results.params[0], alpha_ci95.loc[0], alpha_ci95.loc[1]))
ct_results.tvalues

In [22]:
%%capture
# Negative Binomial Regression Dataframe
data=reg_data[['Biketrips', 'Temp_0','Temp_5','Temp_10','Temp_20','Temp_25','Temp_30','Temp_35',
                         'Humidity','Rain','Wind','February','March','April','May','June','July','August',
                          'September','October','November','December','Peaktravel','Holidays','Weekend',
                          'Visibility']]

#Divide the dataframe into a train and test dataframe
mask = np.random.rand(len(data)) < 0.8
data_train = data[mask]
data_test = data[~mask]
#print('Training data set length='+str(len(poisson_dataset_train)))
#print('Testing data set length='+str(len(poisson_dataset_test)))

#Setup the Ngeative Binomial regression expression in patsy notation. We are telling patsy that Biketrips is our dependent variable and it depends on the other categorical independent regression variables
formula = """Biketrips ~ C(Temp_0)  + C(Temp_5) + C(Temp_10) + C(Temp_20) + C(Temp_25) + C(Temp_30)
                + C(Temp_35) + C(Humidity) + C(Rain) + C(Wind) +C(February)+C(March)+C(April)
                +C(May)+C(June)+C(July)+C(August)+C(September)+C(October)+C(November)+C(December)
                +C(Peaktravel)+C(Holidays)+C(Weekend)+C(Visibility)"""

#Set up the X and Y matrices for the training and testing data sets
y_train, X_train = dmatrices(formula, data_train, return_type='dataframe')
y_test, X_test = dmatrices(formula, data_test, return_type='dataframe')

# providing a value for the alpha term and train the Negative Binomial model on the training dataset
results = sm.GLM(y_train, X_train, sm.families.NegativeBinomial(alpha=0.263)).fit()
print(results.summary())

#Take the exponential of the estimated parameters to generate the Incidence Rate Ratio (IRR)
for i in range(len(results.params)):
    print(math.exp(results.params[i]))
    
#Fitted means from Negative Binomial Regression 
yhat=results.mu

In [23]:
%%capture
#Plot the fitted bike trip counts versus the Pearson residuals for the train data
fig, ax = plt.subplots()
ax.scatter(yhat, results.resid_pearson, c='#d62728')

ax.set_title('Residual vs Mean')
ax.set_ylabel('Pearson residuals')
ax.set_xlabel('Fitted means')

In [24]:
%%capture
#Make predictions on the test data using the trained Negative Binomial regression model
predictions = results.get_prediction(X_test)
predictions_summary= predictions.summary_frame()
print(predictions_summary)

predicted_biketrips=predictions_summary['mean']
actual_biketrips = y_test['Biketrips']

#Plot the predicted counts versus the actual counts for the test data
import seaborn as sns
ax=f, axes = plt.subplots(1, 0, figsize=(10, 6), sharex=False)
sns.distplot(predicted_biketrips, hist=False, color="#d62728", label='Predicted bike trips', kde=True, kde_kws=dict(linewidth=3))
sns.distplot(actual_biketrips, hist=False, color="blue", label='Actual bike trips', kde=True, kde_kws=dict(linewidth=3))

plt.xlabel("Number of bike trips", fontsize=12)
plt.ylabel("Density", fontsize=12)
plt.legend(['Predicted bike trips', 'Actual bike trips'], fontsize=12)
plt.yticks(fontsize=12)